<a href="https://colab.research.google.com/github/11doris/jazz-maestro/blob/colab_word_embeddings/tune_similarity_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tunes as input

In [1]:
!pip uninstall gensim -y

Found existing installation: gensim 3.6.0
Uninstalling gensim-3.6.0:
  Successfully uninstalled gensim-3.6.0


In [2]:
!pip install gensim

     |████████████████████████████████| 24.1 MB 2.4 kB/s 


In [3]:
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pprint
import pandas as pd
import numpy as np
from collections import Counter
import plotly.express as px
from tqdm import tqdm 
from gensim.models.doc2vec import Doc2Vec
import pickle
import os

In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
print(gensim.__version__)

4.1.2


# Initialization

## Download the Data

### Simplified chords, with duplicates

In [6]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy' -O input_seq.txt


--2021-11-18 19:37:46--  https://docs.google.com/uc?export=download&id=1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy
Resolving docs.google.com (docs.google.com)... 209.85.200.102, 209.85.200.113, 209.85.200.138, ...
Connecting to docs.google.com (docs.google.com)|209.85.200.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/iopiqsa1ba4p4b44grbq63phdgigicoe/1637264250000/14329102864480165501/*/1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy?e=download [following]
--2021-11-18 19:37:46--  https://doc-10-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/iopiqsa1ba4p4b44grbq63phdgigicoe/1637264250000/14329102864480165501/*/1YvpsEo6OGObpFsxgfZ_nPtgs1u59ydJy?e=download
Resolving doc-10-4c-docs.googleusercontent.com (doc-10-4c-docs.googleusercontent.com)... 172.253.119.132, 2607:f8b0:4001:c23::84
Connecting to doc-10-4c-docs.googleusercontent.com (doc-10-4c-

### Major Triads, Minor Triads and Dom7 chords:

In [7]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd' -O input_seq.txt

--2021-11-18 19:37:46--  https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd
Resolving docs.google.com (docs.google.com)... 64.233.191.102, 64.233.191.113, 64.233.191.139, ...
Connecting to docs.google.com (docs.google.com)|64.233.191.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/c7phdgqbsq1l0kapj3s0k0ks0hgjqs25/1637264250000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download [following]
--2021-11-18 19:37:47--  https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/c7phdgqbsq1l0kapj3s0k0ks0hgjqs25/1637264250000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download
Resolving doc-08-4c-docs.googleusercontent.com (doc-08-4c-docs.googleusercontent.com)... 172.253.119.132, 2607:f8b0:4001:c23::84
Connecting to doc-08-4c-docs.googleusercontent.com (doc-08-4c-

### Simplified chords, no repetitions

In [8]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=18aqk_RtyCssrtFljjGo9icwgTYXcmKck' -O input_seq.txt


--2021-11-18 19:37:47--  https://docs.google.com/uc?export=download&id=18aqk_RtyCssrtFljjGo9icwgTYXcmKck
Resolving docs.google.com (docs.google.com)... 209.85.200.138, 209.85.200.100, 209.85.200.101, ...
Connecting to docs.google.com (docs.google.com)|209.85.200.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/iedg8dhb1mcavhe07fm16feqoq5sarnq/1637264250000/14329102864480165501/*/18aqk_RtyCssrtFljjGo9icwgTYXcmKck?e=download [following]
--2021-11-18 19:37:48--  https://doc-0k-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/iedg8dhb1mcavhe07fm16feqoq5sarnq/1637264250000/14329102864480165501/*/18aqk_RtyCssrtFljjGo9icwgTYXcmKck?e=download
Resolving doc-0k-4c-docs.googleusercontent.com (doc-0k-4c-docs.googleusercontent.com)... 172.253.119.132, 2607:f8b0:4001:c23::84
Connecting to doc-0k-4c-docs.googleusercontent.com (doc-0k-4c-

### Basic Chords, no repetitions

In [9]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1t12yHtoRp0jMn8nbaEeg3AX4LHgBMTZe' -O input_seq.txt


--2021-11-18 19:37:48--  https://docs.google.com/uc?export=download&id=1t12yHtoRp0jMn8nbaEeg3AX4LHgBMTZe
Resolving docs.google.com (docs.google.com)... 209.85.200.102, 209.85.200.113, 209.85.200.138, ...
Connecting to docs.google.com (docs.google.com)|209.85.200.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/brmv5g5hlo6noj6nv85hof3dg1t7bppv/1637264250000/14329102864480165501/*/1t12yHtoRp0jMn8nbaEeg3AX4LHgBMTZe?e=download [following]
--2021-11-18 19:37:48--  https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/brmv5g5hlo6noj6nv85hof3dg1t7bppv/1637264250000/14329102864480165501/*/1t12yHtoRp0jMn8nbaEeg3AX4LHgBMTZe?e=download
Resolving doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-docs.googleusercontent.com)... 172.253.119.132, 2607:f8b0:4001:c23::84
Connecting to doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-

### Meta Data

In [10]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q' -O input_meta.csv

--2021-11-18 19:37:49--  https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q
Resolving docs.google.com (docs.google.com)... 64.233.182.139, 64.233.182.101, 64.233.182.138, ...
Connecting to docs.google.com (docs.google.com)|64.233.182.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mnvo7s4vk96hr3ntho8c6dn8vtl13kcd/1637264250000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download [following]
--2021-11-18 19:37:50--  https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mnvo7s4vk96hr3ntho8c6dn8vtl13kcd/1637264250000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download
Resolving doc-14-4c-docs.googleusercontent.com (doc-14-4c-docs.googleusercontent.com)... 172.253.119.132, 2607:f8b0:4001:c23::84
Connecting to doc-14-4c-docs.googleusercontent.com (doc-14-4c-

### Read Data

In [11]:
path_to_file = '/content/input_seq.txt'

with open(path_to_file) as f: 
  lines = f.read().splitlines()

In [12]:
data = [line.split(' ') for line in lines]

### Meta Data

In [13]:
meta = pd.read_csv('input_meta.csv', sep='\t')
meta.columns

Index(['id', 'file_name', 'title', 'composer', 'year', 'year_truncated',
       'tonality', 'tune_key', 'tune_mode', 'structure', 'num_bars',
       'time_signature', 'cycle_fifths_order', 'style', 'musicbrainz_id',
       'wikidata_id', 'wikidata_allmusic', 'wiki_link', 'wikidata_description',
       'lyricist'],
      dtype='object')

In [14]:
titles = meta.loc[:, ['id', 'title', 'tune_mode']].drop_duplicates()
titles.set_index('id', inplace=True)

In [15]:
assert len(data) == len(titles)
print(len(data))

1812


In [16]:
titles_dict = titles.to_dict()
index_to_title = titles_dict['title']
index_to_title[1170]

'These Foolish Things'

In [17]:
title_to_index = {value: index for index, value in index_to_title.items()}
title_to_index['These Foolish Things']

1170

# Visualize Chord N-Grams



In [18]:
def ngrams(tokens, n=2, sep='-'):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]

In [19]:
def list_to_ngrams(data, n=2, sep='-'):
  out = []
  for row in data:
    out.append(ngrams(row, n, sep))

  return out

In [20]:
data_ngrams = list_to_ngrams(data, 3)

In [21]:
def raw_chords_to_df(tunes):
  tunes_chords = [item for tune in tunes for item in tune]
  counts = Counter(tunes_chords)
  df = pd.DataFrame(counts.items(),
                    columns=['chord', 'count']).sort_values(by='count', ascending=False)

  return df

In [22]:
df_chords = raw_chords_to_df(data_ngrams)
df_chords.head(10)

,chord,count
14,Dm7-G7-C,3461
109,A7-Dm7-G7,1558
67,D7-G7-C,1164
34,Em7-A7-Dm7,1153
62,C-G7-C,859
155,A7-D7-G7,782
59,Am7-Dm7-G7,675
61,G7-C-G7,645
64,C-C7-F,639
6,Gm7-C7-F,625


In [23]:
df_chords = pd.DataFrame.from_dict(df_chords)
df_chords.sort_values(by=['count'], ascending=False, inplace=True)
df_chords_top = df_chords.query('count > 100')

fig = px.bar(df_chords_top, x='chord', y='count', log_y=True)
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

2021-11-18 19:37:50,840 : INFO : NumExpr defaulting to 2 threads.


# List Tunes for Test Data

In [24]:
contrafacts = {
               "26-2": "Confirmation",
               "52nd Street Theme": "I Got Rhythm",
               "Ablution": "All The Things You Are",
               "Anthropology": "I Got Rhythm",
               "Ballade": "As Long As I Live",
               "Bright Mississippi": "Sweet Georgia Brown",
               "C.T.A.": "I Got Rhythm",
               "Celia": "I Got Rhythm",
               "Cottontail": "I Got Rhythm",
               "Countdown": "Tune Up",
               "Dewey Square": "Oh, Lady Be Good",
               "Dexterity": "I Got Rhythm",
               "Dig": "Sweet Georgia Brown",
               "Donna Lee": "Indiana (Back Home Again In)",
               "Don't Be That Way": "I Got Rhythm",
               "Eternal Triangle": "I Got Rhythm",
               "Evidence": "Just You, Just Me",
               "Flintstones": "I Got Rhythm",
               "Four On Six": "Summertime",
               "Freight Train": "Blues For Alice",
               "Good Bait": "I Got Rhythm",
               "Hackensack": "Oh, Lady Be Good",
               "Half Nelson": "Lady Bird",
               "Hot House": "What Is This Thing Called Love",
               "Impressions": "So What",
               "In A Mellow Tone (In A Mellotone)": "Rose Room",
               "In Walked Bud": "Blue Skies",
               "Ko Ko": "Cherokee",
               "Lennie's Pennies": "Pennies From Heaven",   ## Lennie's Pennies is in minor and therefore transposed to Amin... not possible to recognize like that
               # "Let's Call This": "Honeysuckle Rose",
               "Little Rootie Tootie": "I Got Rhythm",
               "Little Willie Leaps": "All God's Chillun Got Rhythm",
               "Lullaby Of Birdland": "Love Me Or Leave Me",
               "Moose The Mooche": "I Got Rhythm",
               "My Little Suede Shoes": "Jeepers Creepers",
               "Oleo": "I Got Rhythm",
               "Ornithology": "How High The Moon",
               "Passport": "I Got Rhythm",
               "Quasimodo (Theme)": "Embraceable You",
               "Rhythm-a-ning": "I Got Rhythm",
               "Room 608": "I Got Rhythm",
               "Salt Peanuts": "I Got Rhythm",
               "Satellite": "How High The Moon",
               "Scrapple From The Apple": "Honeysuckle Rose", # A section
               "Scrapple From The Apple": "I Got Rhythm", # B section
               #"Segment": "I Got Rhythm",
               "Seven Come Eleven": "I Got Rhythm",
               "Shaw 'Nuff": "I Got Rhythm",
               "Theme, The": "I Got Rhythm",
               "Tour De Force": "Jeepers Creepers",
               "Wow": "You Can Depend On Me",
               "Yardbird Suite": "Rosetta",
               # following tunes are not from wikipedia:
               "Sweet Sue, Just You": "Honeysuckle Rose",  # A section
               # "All Of Me": "Pennies From Heaven", # bars 25-28 of All of Me are same as bars 17-20 of Pennies From Heaven, but different key!
               "Sweet Sue, Just You": "Bye Bye Blackbird", # Bridge same
               "These Foolish Things": "Blue Moon", # first 8 bars same
               
}

contrafacts_test = []
for reference, compare in contrafacts.items():
  contrafacts_test.append([title_to_index[reference], title_to_index[compare]])
  

In [25]:
contrafacts_test[:5]

[[0, 251], [3, 1527], [30, 63], [80, 1527], [104, 1372]]

# TF-IDF

## Build Model

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [30]:
tfidf_ngrams = [1, 2, 3, 4]

tfidf_model = {}
tfidf_vectorizer = {}

for n in tfidf_ngrams:

  data_ngrams = list_to_ngrams(data, n=n)
  print(data_ngrams[0])
  data_tfidf = [" ".join(i) for i in data_ngrams]

  # make sure that no processing on the chord symbols is done!
  tv = TfidfVectorizer(min_df = 0., 
                     max_df = 1., 
                     norm = 'l2', 
                     use_idf = True,
                     stop_words=None,
                     lowercase=False,
                     token_pattern='\\b[\\w#]+')

  tfidf_vectorizer[n] = tv
  tfidf_model[n] = tv.fit_transform(data_tfidf)

['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'Gm7', 'C7', 'Bm7', 'E7', 'A', 'C7', 'F', 'Bbm7', 'Eb7', 'G#', 'Dm7', 'G7', 'C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm7', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C']
['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Gm7', 'Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm7', 'Bbm7-Eb7', 'Eb7-G#', 'G#-Dm7', 'Dm7-G7', 'G7-C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm7', 'Gm7-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C']
['C-Eb7-G#', 'Eb7-G#-B7', 'G#-B7-E', 'B7-E-G7', 'E-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-F', 'C7-F-G#7', 'F-G#7-C#', 'G#7-C#-E7', 'C#-E7-

In [35]:
for i in range(len(tfidf_vectorizer)):
  print(tfidf_vectorizer[i+1].get_feature_names_out())

['A' 'A7' 'Am' 'Am7' 'B' 'B7' 'Bb' 'Bb7' 'Bbm' 'Bbm7' 'Bm' 'Bm7' 'C' 'C#'
 'C#7' 'C#m' 'C#m7' 'C7' 'Cm' 'Cm7' 'D' 'D7' 'Dm' 'Dm7' 'E' 'E7' 'Eb'
 'Eb7' 'Ebm' 'Ebm7' 'Em' 'Em7' 'F' 'F#' 'F#7' 'F#m' 'F#m7' 'F7' 'Fm' 'Fm7'
 'G' 'G#' 'G#7' 'G#m' 'G#m7' 'G7' 'Gm' 'Gm7']
['A' 'A7' 'Am' 'Am7' 'B' 'B7' 'Bb' 'Bb7' 'Bbm' 'Bbm7' 'Bm' 'Bm7' 'C' 'C#'
 'C#7' 'C#m' 'C#m7' 'C7' 'Cm' 'Cm7' 'D' 'D7' 'Dm' 'Dm7' 'E' 'E7' 'Eb'
 'Eb7' 'Ebm' 'Ebm7' 'Em' 'Em7' 'F' 'F#' 'F#7' 'F#m' 'F#m7' 'F7' 'Fm' 'Fm7'
 'G' 'G#' 'G#7' 'G#m' 'G#m7' 'G7' 'Gm' 'Gm7']
['A' 'A7' 'Am' 'Am7' 'B' 'B7' 'Bb' 'Bb7' 'Bbm' 'Bbm7' 'Bm' 'Bm7' 'C' 'C#'
 'C#7' 'C#m' 'C#m7' 'C7' 'Cm' 'Cm7' 'D' 'D7' 'Dm' 'Dm7' 'E' 'E7' 'Eb'
 'Eb7' 'Ebm' 'Ebm7' 'Em' 'Em7' 'F' 'F#' 'F#7' 'F#m' 'F#m7' 'F7' 'Fm' 'Fm7'
 'G' 'G#' 'G#7' 'G#m' 'G#m7' 'G7' 'Gm' 'Gm7']
['A' 'A7' 'Am' 'Am7' 'B' 'B7' 'Bb' 'Bb7' 'Bbm' 'Bbm7' 'Bm' 'Bm7' 'C' 'C#'
 'C#7' 'C#m' 'C#m7' 'C7' 'Cm' 'Cm7' 'D' 'D7' 'Dm' 'Dm7' 'E' 'E7' 'Eb'
 'Eb7' 'Ebm' 'Ebm7' 'Em' 'Em7' 'F' 'F#' 'F#7' 'F#m' 'F#m7' 'F

In [39]:
i = 1

tv_matrix = tfidf_model[i].toarray()

vocab = tfidf_vectorizer[i].get_feature_names_out()
tfidf_chords = pd.DataFrame(tv_matrix, columns = vocab)
tfidf_chords

,A,A7,Am,Am7,B,B7,Bb,Bb7,Bbm,Bbm7,Bm,Bm7,C,C#,C#7,C#m,C#m7,C7,Cm,Cm7,D,D7,Dm,Dm7,E,E7,Eb,Eb7,Ebm,Ebm7,Em,Em7,F,F#,F#7,F#m,F#m7,F7,Fm,Fm7,G,G#,G#7,G#m,G#m7,G7,Gm,Gm7
0,0.096717,0.000000,0.000000,0.045210,0.0,0.290675,0.000000,0.000000,0.0,0.095548,0.0,0.061840,0.151813,0.094245,0.000000,0.00000,0.0,0.205434,0.000000,0.000000,0.0,0.037845,0.000000,0.071119,0.437645,0.088860,0.000000,0.462625,0.000000,0.0,0.000000,0.000000,0.159336,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.517709,0.065885,0.0,0.000000,0.207987,0.0,0.236796
1,0.000000,0.108299,0.000000,0.133066,0.0,0.000000,0.000000,0.000000,0.0,0.281223,0.0,0.182013,0.000000,0.554777,0.000000,0.00000,0.0,0.000000,0.000000,0.259732,0.0,0.000000,0.000000,0.104661,0.000000,0.130770,0.253959,0.000000,0.000000,0.0,0.000000,0.137777,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.609573,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
2,0.000000,0.000000,0.000000,0.262650,0.0,0.168868,0.256757,0.000000,0.0,0.000000,0.0,0.359263,0.000000,0.547519,0.250245,0.00000,0.0,0.000000,0.000000,0.256333,0.0,0.000000,0.000000,0.000000,0.127125,0.322647,0.000000,0.000000,0.000000,0.0,0.000000,0.067987,0.000000,0.0,0.0,0.000000,0.190105,0.164711,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.302067,0.000000,0.0,0.000000
3,0.000000,0.000000,0.000000,0.524170,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.547596,0.000000,0.000000,0.00000,0.0,0.052929,0.000000,0.000000,0.0,0.048753,0.000000,0.412280,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.051315,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.497592,0.0,0.000000
4,0.000000,0.121667,0.000000,0.000000,0.0,0.000000,0.487119,0.179672,0.0,0.000000,0.0,0.000000,0.033465,0.000000,0.000000,0.00000,0.0,0.498137,0.000000,0.000000,0.0,0.041712,0.000000,0.000000,0.000000,0.000000,0.095102,0.000000,0.479610,0.0,0.000000,0.000000,0.043904,0.0,0.0,0.278018,0.000000,0.187494,0.000000,0.000000,0.0,0.000000,0.290469,0.0,0.000000,0.163742,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,0.000000,0.000000,0.000000,0.000000,0.0,0.208837,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.436283,0.000000,0.000000,0.64808,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.255479,0.157214,0.000000,0.000000,0.000000,0.130264,0.0,0.169934,0.000000,0.071547,0.0,0.0,0.151021,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.426940,0.0,0.000000
1808,0.000000,0.322223,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.199416,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.082853,0.000000,0.544947,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.307445,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.432721,0.0,0.000000,0.520388,0.0,0.000000
1809,0.000000,0.262272,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.324628,0.000000,0.000000,0.00000,0.0,0.000000,0.339845,0.000000,0.0,0.134875,0.704728,0.190096,0.000000,0.237516,0.000000,0.000000,0.000000,0.0,0.000000,0.166829,0.070982,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.264730,0.0,0.000000
1810,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.734458,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.262790,0.0,0.0,0.000000,0.000000,0.498778,0.000000,0.000000,0.0,0.189744,0.000000,0.0,0.000000,0.326695,0.0,0.000000


The `tfidf_model` is a list of the 4 models fitted with ngram size 1...4. 
Each of the model contains a sparse matrix with tunes being in the rows and the vocabulary being in the columns.

In [41]:
tfidf_model[1][100]

<1x48 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>

In [42]:
tfidf_model

{1: <1812x48 sparse matrix of type '<class 'numpy.float64'>'
 	with 21387 stored elements in Compressed Sparse Row format>,
 2: <1812x48 sparse matrix of type '<class 'numpy.float64'>'
 	with 21387 stored elements in Compressed Sparse Row format>,
 3: <1812x48 sparse matrix of type '<class 'numpy.float64'>'
 	with 21387 stored elements in Compressed Sparse Row format>,
 4: <1812x48 sparse matrix of type '<class 'numpy.float64'>'
 	with 21378 stored elements in Compressed Sparse Row format>}

## Test of the Contrafacts tunes

In [43]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2']
for c in tfidf_model.keys():
  cols.append(f"cosval_ngram{c}")
df_eval_tfidf = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  for ngram_size, tfidf_m in tfidf_model.items():
    row.append(cosine_similarity(tfidf_m[tune1], tfidf_m[tune2])[0][0])
  
  df_eval_tfidf.loc[len(df_eval_tfidf)] = row

df_eval_tfidf

,Tune1,Tune2,cosval_ngram1,cosval_ngram2,cosval_ngram3,cosval_ngram4
0,26-2,Confirmation,0.500697,0.492477,0.494181,0.501448
1,52nd Street Theme,I Got Rhythm,0.831848,0.825161,0.817133,0.809436
2,Ablution,All The Things You Are,0.917601,0.914305,0.916288,0.925313
3,Anthropology,I Got Rhythm,0.715753,0.706849,0.709068,0.703962
4,Ballade,As Long As I Live,0.638264,0.634892,0.623167,0.612081
5,Bright Mississippi,Sweet Georgia Brown,0.898530,0.891729,0.881984,0.870316
6,C.T.A.,I Got Rhythm,0.543100,0.535901,0.536527,0.539971
7,Celia,I Got Rhythm,0.362829,0.343311,0.344947,0.337100
8,Cottontail,I Got Rhythm,0.926148,0.923451,0.920177,0.916899
9,Countdown,Tune Up,0.674910,0.640808,0.639790,0.627937


In [44]:
print(tfidf_model[2][1171].toarray())

[[0.         0.21676466 0.         0.46608648 0.         0.0856189
  0.         0.08002741 0.         0.         0.         0.
  0.29065988 0.         0.         0.         0.         0.18153323
  0.         0.         0.         0.22294609 0.         0.5237075
  0.         0.         0.         0.         0.         0.
  0.         0.20682314 0.         0.         0.         0.
  0.09638605 0.         0.09923582 0.         0.         0.
  0.         0.         0.         0.45947178 0.         0.0871858 ]]


In [45]:
m = tfidf_model[1]
tfidf_cos = cosine_similarity(m)

In [46]:
from operator import itemgetter

tfidf_dict = {}
for row_index, row in enumerate(tfidf_cos):
  indices, L_sorted = zip(*sorted(enumerate(row), key=itemgetter(1), reverse=True))
  
  # do not report self-similiarity, therefore remove the reference tune
  remove_index = indices.index(row_index)
  
  indices = list(indices)
  indices.pop(remove_index)
  L_sorted = list(L_sorted)
  L_sorted.pop(remove_index)

  tfidf_dict[row_index] = {}
  tfidf_dict[row_index]['tune_indices'] = indices[:10]
  tfidf_dict[row_index]['score'] = [round(num, 3) for num in L_sorted[:10]] 

tfidf_dict

{0: {'score': [0.853,
   0.821,
   0.745,
   0.725,
   0.711,
   0.664,
   0.643,
   0.633,
   0.629,
   0.618],
  'tune_indices': [425, 262, 124, 1236, 1011, 1307, 1199, 549, 778, 1639]},
 1: {'score': [0.856,
   0.752,
   0.727,
   0.707,
   0.682,
   0.668,
   0.667,
   0.619,
   0.615,
   0.604],
  'tune_indices': [1005, 1194, 745, 395, 1294, 338, 115, 906, 173, 1322]},
 2: {'score': [0.745,
   0.699,
   0.694,
   0.664,
   0.63,
   0.621,
   0.619,
   0.619,
   0.615,
   0.598],
  'tune_indices': [115, 350, 379, 153, 441, 1083, 113, 1322, 1189, 1326]},
 3: {'score': [0.985,
   0.963,
   0.963,
   0.957,
   0.953,
   0.953,
   0.952,
   0.93,
   0.93,
   0.923],
  'tune_indices': [516, 767, 308, 318, 651, 835, 391, 632, 92, 937]},
 4: {'score': [0.675,
   0.622,
   0.618,
   0.599,
   0.585,
   0.584,
   0.583,
   0.581,
   0.579,
   0.579],
  'tune_indices': [1141, 55, 735, 384, 237, 107, 1574, 1313, 94, 138]},
 5: {'score': [0.755,
   0.748,
   0.741,
   0.739,
   0.73,
   0.72,


In [47]:
tfidf_file = 'tfidf_cossim.pickle'
with open(tfidf_file, 'wb') as f:
      # Pickle the 'data' dictionary using the highest protocol available.
      pickle.dump(tfidf_dict, f, pickle.HIGHEST_PROTOCOL)

from google.colab import files
files.download(tfidf_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Doc2Vec Embeddings

## Read and Tag Data

In [ ]:
import smart_open

def read_corpus(fname, ngram_size, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = ngrams(line.split(), ngram_size, '-')
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

## Doc2Vec Hyperparameters

In [ ]:
!rm -R models_doc2vec
!rm -R traindata_doc2vec

In [ ]:
!mkdir models_doc2vec
!mkdir traindata_doc2vec

In [ ]:
import multiprocessing

model_path = './models_doc2vec'
model_prefix = 'autos'
data_path = './traindata_doc2vec'

param_grid = {'d2v': {'variant': ['pv-dm', 
                                  'concat',
                                  'pv-dbow'
                                  ], 
                      'window': [
                                 2, 
                                 3, 
                                 #4, 
                                 #10
                                 ],
                      'sample': [0.1],
                      'ngrams': [
                                 1, 
                                 2, 
                                 #3, 
                                 #4
                                 ],
                     }
              }


common_kwargs = dict(
    vector_size=100, 
    epochs=20, 
    min_count=2,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=10, 
    hs=0,
)

for algo, params in param_grid.items():
    for variant in params['variant']:
        dm = 0 if variant == 'pv-dbow' else 1
        concat = 1 if variant ==  'concat' else 0

        for ngram_size in params['ngrams']:
            
            # prepare data with the provided ngram size
            train_corpus = list(read_corpus(path_to_file, ngram_size=ngram_size))

            # evaluate different window sizes
            for window in params['window']:

                # evaluate the effect of different sample sizes
                for sample in params['sample']:

                    # calculate the model
                    if algo == 'd2v':
                        print(f"Model: {variant}, dm={dm}, dm_concat={concat}; ngrams={ngram_size}, window={window}, sample={sample}")
                        print(f"Corpus: {train_corpus[0]}")
                        model = gensim.models.doc2vec.Doc2Vec(**common_kwargs,
                                                              dm = dm,
                                                              dm_concat = concat,
                                                              )
                        model.build_vocab(train_corpus)
                        model.train(train_corpus, 
                                    total_examples=model.corpus_count, 
                                    epochs=model.epochs)

                    else:
                        model = gensim.models.FastText(data_ngrams, vector_size=size, window=window, sg=sg, epochs=50)

                    file_name = f"{model_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    model.save(file_name + '.model')

                    file_name = f"{data_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    with open(file_name + '.train', 'wb') as f:
                          # Pickle the 'data' dictionary using the highest protocol available.
                          pickle.dump(train_corpus, f, pickle.HIGHEST_PROTOCOL)

### Trial with Models according to the Original Paper 

In [ ]:
import multiprocessing
from collections import OrderedDict

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

train_corpus = list(read_corpus(path_to_file, ngram_size=1))

common_kwargs = dict(
    vector_size=100, 
    epochs=50, 
    min_count=2,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=5, 
    hs=0,
)

simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, **common_kwargs),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, window=10, alpha=0.05, comment='alpha=0.05', **common_kwargs),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, window=5, **common_kwargs),
]

for model in simple_models:
    model.build_vocab(train_corpus)
    model.train(train_corpus, 
                total_examples=model.corpus_count, 
                epochs=model.epochs)

    print("%s vocabulary scanned & state initialized" % model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

In [ ]:
!pip install testfixtures

In [ ]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[2]])

In [ ]:
models_by_name

In [ ]:
from scipy import spatial

methods = ['Doc2Vec(dbow,d100,n5,mc2,t2)', 'Doc2Vec(dm/c,d100,n5,w5,mc2,t2)', 'dbow+dmm', 'dbow+dmc']

# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2', 'score_dbow', 'score_concat', 'score_dbow+dmm', 'score_dbow+dmc']

df_eval = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
    row = [index_to_title[tune1], index_to_title[tune2]]

    for m in methods:
        vec1 = models_by_name[m][tune1]
        vec2 = models_by_name[m][tune2]
      
        row.append(1 - spatial.distance.cosine(vec1, vec2))
  
    df_eval.loc[len(df_eval)] = row

df_eval

### End of Other Trial

In [ ]:
from gensim.models import KeyedVectors

from os import listdir
from os.path import isfile, join

names = [f for f in listdir(model_path) if isfile(join(model_path, f))]
print(names)
models = {}

for name in names:
    file_name = f"{model_path}/{name}"
    print(file_name)
    models[name] = Doc2Vec.load(file_name)

In [ ]:
names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print(names)
data = {}

for name in names:
    file_name = f"{data_path}/{name}"
    print(file_name)
    with open(file_name, 'rb') as f:
        data[name] = pickle.load(f)

In [ ]:
for name, m in models.items():
  print(m)

In [ ]:
def compare_doc_models(models, **kwargs):

    passes = 1

    df = pd.DataFrame()
    names = []
    scores = []
    runs = []

    for p in range(passes):
      print(f"\n*** Run {p}")
      for name, model in models.items():
        ranks = []
        second_ranks = []
        similarities = []

        print(name)

        # load the training corpus data
        train_name = os.path.splitext(name)[0] + '.train'
        train_corpus = data[train_name]
        print(train_corpus[:5])

        # use the model to infer the similarity for a tune
        for doc_id in range(len(train_corpus)):
            inferred_vector = model.infer_vector(train_corpus[doc_id].words)
            sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
            similarities.append(sims)
            rank = [docid for docid, sim in sims].index(doc_id)
            ranks.append(rank)

            second_ranks.append(sims[1])

        counter = Counter(ranks)
        print(counter)

        tunes_sim = counter[0]
        tunes_not_sim = len(train_corpus) - counter[0]
        score = 100 * tunes_not_sim/(tunes_sim+tunes_not_sim)

        print(f"\tTunes similar to themselves in first place: {tunes_sim}")
        print(f"\tTunes not similar in first place: {tunes_not_sim}")
        print(f"\t% of tunes not similar to themselves in first place: {score:.2f}%")

        names.append(name)
        scores.append(score)
        runs.append(p)
      
    df = pd.DataFrame({'name': names, 
                        'score': scores,
                        'pass': runs
                      })

    return df

In [ ]:
df_score = compare_doc_models(models)

In [ ]:
pd.pivot_table(df_score, index = 'name', values = 'score', columns='pass')

In [ ]:
def compare_doc_models2(models, **kwargs):

    passes = 1

    df = pd.DataFrame()
    topn = 5

    names = []
    scores = []
    runs = []
    for p in range(passes):
      print(f"\n*** Run {p}")
      for name, model in models.items():
        tune_sim = 0

        print(name)

        # load the training corpus data
        train_name = os.path.splitext(name)[0] + '.train'
        train_corpus = data[train_name]
        print(train_corpus[:5])

        # use the model to infer the similarity for a tune
        for doc_id in range(len(train_corpus)):
            inferred_vector = model.infer_vector(train_corpus[doc_id].words)
            sims = model.dv.most_similar([inferred_vector], topn=topn)
            if doc_id in [key for key, value in sims]:
              tune_sim += 1

        tunes_not_sim = len(train_corpus) - tune_sim
        score = 100 * tunes_not_sim/(tune_sim+tunes_not_sim)

        print(f"\tTunes similar to themselves in top {topn} place: {tune_sim}")
        print(f"\tTunes not similar in first place: {tunes_not_sim}")
        print(f"\t% of tunes not similar to themselves in first place: {score:.2f}%")

        names.append(name)
        scores.append(score)
        runs.append(p)
      
    df = pd.DataFrame({'name': names, 
                        'score': scores,
                        'pass': runs
                      })

    return df

In [ ]:
df_score2 = compare_doc_models2(models)

In [ ]:
pd.pivot_table(df_score2, index = 'name', values = 'score', columns='pass')

In [ ]:
import time
t = time.localtime()
timestamp = time.strftime('%Y%m%d_%H%M%S', t)
print(timestamp)

In [ ]:
score_filename = f'score_{timestamp}.csv'
df_score.to_csv(score_filename)

In [ ]:
 !zip -r models.zip models_doc2vec/ 

In [ ]:
from google.colab import files
files.download("models.zip")

In [ ]:
files.download(score_filename)

### Assess the best Model

In [ ]:
#best_model = 'autos_d2v_pv-dbow_3_4_0.1'
best_model = 'autos_d2v_concat_1_3_0.1'

In [ ]:
model = models[f"{best_model}.model"]

#### Assess self-similarity

In [29]:
ranks = []
second_ranks = []
similarities = []

train_corpus = data[f"{best_model}.train"]

for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    similarities.append(sims)
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

NameError: ignored

In [ ]:
import collections

counter = collections.Counter(ranks)
print(counter)

tunes_sim = counter[0]
tunes_not_sim = len(train_corpus) - counter[0]

print()
print(f"Tunes similar to themselves in first place: {tunes_sim}")
print(f"Tunes not similar in first place: {tunes_not_sim}")
print(f"% of tunes not similar to themselves in first place: {100 * tunes_not_sim/(tunes_sim + tunes_not_sim):.2f}%")

In [ ]:
print('Tunes that are similar to other tunes before they are similar to themselves:')
for tune_index, value in enumerate(ranks):
  if value > 0:
    print(f"'{titles.iloc[tune_index, 0]}': {value}")

#### Determine Similarity of Tunes

In [ ]:
assert(len(model.dv.vectors == len(list(title_to_index.keys()))))

In [ ]:
tunes_eval_list = [
                   'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'Hundred Years From Today, A',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

In [ ]:
for tune in tunes_eval_list:
  sim = model.dv.most_similar(title_to_index[tune])

  print()
  print(f"'{tune}' is most similar to:")
  for index, value in sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')


#### Evaluate Similarity for Jazz Contrafacts



In [ ]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2', 'doc2vec']
df_eval_doc2vec = pd.DataFrame(columns=cols)

# calculate a dataframe with the cos sim value for the test tunes
for tune1, tune2 in contrafacts_test:
  row = [index_to_title[tune1], 
         index_to_title[tune2]]
  row.append(model.dv.similarity(tune1, tune2))
  
  df_eval_doc2vec.loc[len(df_eval_doc2vec)] = row

df_eval_doc2vec

##### Compare results from TF-IDF and doc2vec



In [ ]:
df_eval_tfidf.merge(df_eval_doc2vec)

##### Verbose Testing Output

In [ ]:
similarity_found = 0

for tune1, tune2 in contrafacts.items():
  index1 = title_to_index[tune1]
  index2 = title_to_index[tune2]

  sim = model.dv.similarity(index1, index2)
  most_sim = model.dv.most_similar(index1, topn=5)
  
  # check whether the index2 was found among the topn similar tunes
  for i in most_sim:
    if index2 == i[0]:
      similarity_found += 1

  print()
  print(f"'{tune1}' to '{tune2}': {sim}")
  print(f"'{tune1}' is most similar to:")
  for index, value in most_sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')


print()
num_tunes = len(contrafacts)
print(f"Number of tunes tested: {num_tunes}")
print(f"Number of similarities found: {similarity_found}")
print(f"Result: {100 * similarity_found / num_tunes}%")

In [ ]:
model.dv.similarity(0, 1)

#### Download Model

In [ ]:
from google.colab import files
files.download(f"{model_path}/{best_model}.model")
files.download(f"{data_path}/{best_model}.train")